In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt

import os

import random
import cv2

import pathlib

In [11]:
ds_path = "/Users/mantesssa/Projects/nn/nn_tf/BirdVsDrone/"
data_dir = pathlib.Path(ds_path)

image_count = len(list(data_dir.glob('*/*.*')))
# print(image_count) # 828

tree = os.walk(ds_path)

fulllist =[]
for i in tree:
    fulllist.append(i)
    
listofdronesnames = []
for i in fulllist[1][2]:
    if i[-4:] =='jpeg' or i[-4:] =='JPEG' or i[-3:] =='png':
        listofdronesnames.append(i) 
listofdroneslabels = [0]*len(listofdronesnames)

listofbirdsnames = []
for i in fulllist[2][2]:
    if i[-4:] =='jpeg' or i[-4:] =='JPEG' or i[-3:] =='png':
        listofbirdsnames.append(i) # .DS_Store ???
listofbirdslabels = [1]*len(listofbirdsnames)


fulllistnames = listofdronesnames + listofbirdsnames
fulllistlabels = listofdroneslabels + listofbirdslabels
print(len(fulllistnames)) # 825 


list_of_img_matrixes = []

for i in range(len(fulllistlabels)): # len = 825
    if fulllistlabels[i] == 0:
        classnamed = "Drones"
    else:
        classnamed = "Birds"
    img = cv2.imread(ds_path + classnamed + "/" + fulllistnames[i])
    RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    smallimgb = cv2.resize(RGB_img, (180,180),interpolation=cv2.INTER_CUBIC)
    normsmallimg = smallimgb/ 255
    list_of_img_matrixes.append(normsmallimg)
        
    

listofindexes = []
for i in range(len(fulllistlabels)): # len = 825
    listofindexes.append(i)

random.shuffle(listofindexes)
shuffledlistofindexes = listofindexes
# print(len(shuffledlistofindexes)) # 800

801


In [12]:
# dataset = 800 indexes from 0 to 799

# 0|   0-39
# 1|   40-79
# 2|   80-119 
# 3|   120-159
# 4|   160-199
# 5|   200-239
# 6|   240-279
# 7|   280-319
# 8|   320-359
# 9|   360-399
# 10|  400-439
# 11|  440-479
# 12|  480-519
# 13|  520-559
# 14|  560-599
# 15|  600-639
# 16|  640-679
# 17|  680-719
# 18|  720-759
# 19|  760-799


class DataGenerator(tf.keras.utils.Sequence):
#     'Generates data for Keras'                      shufled list of indexes
    def __init__(self, fulllistnames, fulllistlabels, list_IDs, is_validation=False, 
                 batch_size=40, dim=(None, 180, 180, 3), n_channels=1, n_classes=2):
        'Initialization'
        self.fulllistnames = fulllistnames
        self.fulllistlabels = fulllistlabels
        self.list_IDs = list_IDs
        self.is_validation = is_validation
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.valid = is_validation
        self.indexes = np.arange(len(self.list_IDs))
        self.number_of_batches = int(np.floor(len(self.list_IDs) / self.batch_size))               # 20
        self.number_of_valid_indexes = 2                                                           # 2
        self.number_of_training_indexes =  self.number_of_batches - self.number_of_valid_indexes   # 18
        self.valind_list_of_indexes = []
        self.training_list_of_indexes = []
        self.valid_index = self.number_of_training_indexes
        self.on_epoch_end()
      
    def __len__(self):
        'Denotes the number of batches per epoch'
        if self.is_validation:
            return self.number_of_valid_indexes # 2
        else:
            return self.number_of_training_indexes # = 18 
 
    def __getitem__(self, index): # index = number of batch from 0 to __len__ 
        if self.is_validation:#if validation set
            indexes = self.valind_list_of_indexes[index*self.batch_size : (index+1)*self.batch_size]      
            list_IDs_temp = [self.list_IDs[k] for k in indexes]
            X, y = self.__data_generation(list_IDs_temp)

        else:#if test set
            indexes = self.training_list_of_indexes[index*self.batch_size : (index+1)*self.batch_size]
            list_IDs_temp = [self.list_IDs[k] for k in indexes]
            X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        if self.valid_index == self.number_of_training_indexes:
            self.valid_index = 0
        else:
            self.valid_index += self.number_of_valid_indexes 
        
        self.indexes = np.arange(len(self.list_IDs))

        self.valind_list_of_indexes = self.indexes[self.valid_index*self.batch_size:(self.valid_index+self.number_of_valid_indexes)*self.batch_size]
        self.training_list_of_indexes = list(set(self.indexes).difference(self.valind_list_of_indexes))

        random.shuffle(self.valind_list_of_indexes)
        random.shuffle(self.training_list_of_indexes)

        # print("self.indexes = ",self.indexes[0:100])
        # print("self.valind_list_of_indexes = ",self.valind_list_of_indexes[0],self.valind_list_of_indexes[-1])
        # print("self.training_list_of_indexes = ",self.training_list_of_indexes[0],self.training_list_of_indexes[-1])

    def __data_generation(self, list_IDs_temp):
        y = []
        X = []
        for ID in (list_IDs_temp):
            classnumber= self.fulllistlabels[ID]
            z = list_of_img_matrixes[ID]
            X.append(z)
            y.append(classnumber) # if drone = 0 if bird = 1
        return np.array(X), keras.utils.to_categorical(y,num_classes=None)
        





In [13]:
allinall = np.array([  #  allinall.shape => (3, 3, 3, 8)  
    [#                       RGB  chanels
    [[0.75, 0.25, 0.25, 0.75, 0.5, 0.5, 1.0, 0.0,]] * 3,      # a11  <= indexes like matrix    first "pixel"    
    [[1.0, 0.0, 0.5, 0.5, 0.75, 0.25, 0.75, 0.25,]] * 3,      # a12
    [[0.75, 0.25, 0.75, 0.25, 1.0, 0.0, 0.5, 0.5,]] * 3,      # a13
    ],
#####
    [
    [[0.5, 0.5, 0.0, 1.0, 0.25, 0.75, 0.75, 0.25,]] * 3,      # a21
    [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,]] * 3,         # a22
    [[0.5, 0.5, 1.0, 0.0, 0.75, 0.25, 0.25, 0.75,]] * 3,      # a23
    ],
#####
    [
    [[0.25, 0.75, 0.25, 0.75, 0.0, 1.0, 0.5, 0.5,]] * 3,      # a31
    [[0.0, 1.0, 0.5, 0.5, 0.25, 0.75, 0.25, 0.75,]] * 3,      # a32
    [[0.25, 0.75, 0.75, 0.25, 0.5, 0.5, 0.0, 1.0,]] * 3,      # a33
    ],
    ], dtype= np.float32)


print(allinall.shape)
# allinall = tf.convert_to_tensor(allinall)
# print(allinall)
initializer = tf.keras.initializers.Constant(allinall)


(3, 3, 3, 8)


In [14]:
from keras import Model

# Defining model input
input_ =  tf.keras.Input(shape=(180, 180, 3))

# Defining first parallel layer
in_1 = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same', name="initializer_on", kernel_initializer=initializer)(input_)
# conv_1 = tf.keras.layers.BatchNormalization()(in_1)
# conv_1 = AveragePooling2D(pool_size=(2, 2), strides=(3, 3))(conv_1)


# Defining second parallel layer
in_2 = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same', name="initializer_off")(input_)

# Concatenating layers
concat = tf.keras.layers.concatenate([in_1, in_2])
# concat = tf.keras.layers.concatenate([conv_1, conv_2])


minimodel = Model(inputs=[input_], outputs=[concat], name="allinall")
# minimodel.summary()

In [21]:
batch_size = 40
img_height = 180
img_width = 180
class_names = ['Drones', 'Birds']
num_classes = len(class_names)

# data_augmentation = keras.Sequential(
#   [
#     layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
#     layers.RandomRotation(0.12),
#     layers.RandomZoom(0.3),
#   ]
# )

# MOBILE_NET_V3
# if alpha = 1.0 => 2kk parameters
# if alpha = 0.5 => 570k parameters
# if alpha = 0.4 => 445k parameters
# if alpha = 0.3 => 327k parameters
# if alpha = 0.2 => 230k parameters
# if alpha = 0.1 => 144k parameters
# if alpha = 0.0 =>  79k parameters


# model = tf.keras.applications.MobileNetV3Small(
# 		input_shape=(180,180,3),
#     alpha=0.0,    
#     minimalistic=False,
# 		include_top=True,
#     weights=None,
#     input_tensor=None,
#     classes=num_classes,
# 		pooling=None,
#     classifier_activation="softmax",
# 		include_preprocessing=True
#     )












# MobileNetV3Small model with data augmentation
base_model = tf.keras.applications.MobileNetV3Small(
		input_shape=(180,180,3),
    alpha=0.4,    
    minimalistic=False,
		include_top=True,
    weights=None,
    input_tensor=None,
    classes=num_classes,
		pooling=None,
    classifier_activation="softmax",
		include_preprocessing=True
    )


inputs = tf.keras.Input(shape=(180, 180, 3))
x = layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3))(inputs)
x = layers.RandomRotation(0.12)(x)
x = layers.RandomZoom(0.3)(x)
x = base_model(x)


model = tf.keras.Model(inputs=inputs, outputs=x)




model.summary() # too big 

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 180, 180, 3)]     0         
                                                                 
 random_flip_10 (RandomFlip)  (None, 180, 180, 3)      0         
                                                                 
 random_rotation_10 (RandomR  (None, 180, 180, 3)      0         
 otation)                                                        
                                                                 
 random_zoom_10 (RandomZoom)  (None, 180, 180, 3)      0         
                                                                 
 MobilenetV3small (Functiona  (None, 2)                450586    
 l)                                                              
                                                                 
Total params: 450,586
Trainable params: 445,146
Non-trainab

In [22]:
def scheduler(epoch, lr):
  if epoch < 25:
    return lr
  else:
    return lr * tf.math.exp(-0.9)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

model.compile(optimizer='adam',
              # loss='binary_crossentropy',
              loss='categorical_crossentropy',
              # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])




In [24]:
#FIT#############################################################################################################

params = {'dim': (None, 180, 180, 3),
          'batch_size': 40,
          'n_classes': 2,
          'n_channels': 1,}

training_generator = DataGenerator(fulllistnames,fulllistlabels,shuffledlistofindexes,False, **params)
validation_generator = DataGenerator(fulllistnames,fulllistlabels,shuffledlistofindexes,True, **params)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/Users/mantesssa/Projects/nn/nn_tf/logs")

epochs = 10 # n*10
history = model.fit(training_generator,
          validation_data=validation_generator,
          epochs=epochs,
          callbacks=[tensorboard_callback,callback])



Epoch 1/10
18/18 [==============================] - 12s 631ms/step - loss: 0.6205 - accuracy: 0.6889 - val_loss: 0.6944 - val_accuracy: 0.4250 - lr: 0.0010
Epoch 2/10
18/18 [==============================] - 10s 560ms/step - loss: 0.5204 - accuracy: 0.7417 - val_loss: 0.6982 - val_accuracy: 0.4375 - lr: 0.0010
Epoch 3/10
 4/18 [=====>........................] - ETA: 7s - loss: 0.4417 - accuracy: 0.7812

KeyboardInterrupt: 

In [ ]:
# model.save("/Users/mantesssa/Projects/nn/nn_tf/my_model_100e_dropout015")# worked saved all model 
# model.save_weights("/Users/mantesssa/Projects/nn/nn_tf/checkpoints/my_checkpoint_dropout015") # worked saved in checkpoints

In [11]:
# model = tf.keras.models.load_model("/Users/mantesssa/Projects/nn/nn_tf/my_model_100e")
# model.summary()

In [14]:
# # summarize filters in each convolutional layer

# summarize filter shapes

for layer in model.layers:
  # check for convolutional layer
  if 'conv' not in layer.name:
    continue
  # get filter weights
  filters, biases = layer.get_weights()
  print(layer.name, filters.shape)

# We can see that all convolutional layers use 3×3 filters, which are small and perhaps easy to interpret.


conv2d (3, 3, 16, 16)
conv2d_1 (3, 3, 16, 32)
conv2d_2 (3, 3, 32, 64)
conv2d_3 (3, 3, 64, 128)


In [15]:
# new_model = tf.keras.models.load_model("/Users/mantesssa/Projects/nn/nn_tf/my_model_100e")
# new_model.summary()

In [ ]:
# new_model = tf.keras.models.load_model("/Users/mantesssa/Projects/nn/nn_tf/my_model_100e")
# new_model.summary()

In [ ]:
######################################################
test_path = "/Users/mantesssa/Projects/nn/nn_tf/test/"    # it works 

tree = os.walk(test_path)

fulllist = []
for i in tree:
    fulllist.append(i)
    
listofdronesnames = []
for i in fulllist[1][2]:
    if i[-4:] =='jpeg' or i[-4:] =='JPEG' or i[-3:] =='png':
        listofdronesnames.append(i)
listofdroneslabels = [0]*len(listofdronesnames)

listofbirdsnames = []
for i in fulllist[2][2]:
    if i[-4:] =='jpeg' or i[-4:] =='JPEG' or i[-3:] =='png':
        listofbirdsnames.append(i) # .DS_Store ???
listofbirdslabels = [1]*len(listofbirdsnames)




fulllistnames = listofdronesnames + listofbirdsnames
fulllistlabels = listofdroneslabels + listofbirdslabels
print(len(fulllistnames)) # 825 
print(len(fulllistlabels)) # 825 


# list_of_test_img_matrixes = []

img_height = 180
img_width = 180

for i in range(len(fulllistlabels)): # len = 825
    if fulllistlabels[i] == 0:
        classnamed = "Drones"
    else:
        classnamed = "Birds"
    img_path = test_path + classnamed + "/" + fulllistnames[i]


    img = tf.keras.utils.load_img(img_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) #  Create a batch


    predictions = new_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    print(img_path)
    print(
    "____ {} ______ with a {:.2f}% confidence."
    # "This image most likely belongs to ____ {} ______ with a {:.2f}% confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
    )


In [89]:
####################################################################################### pre loaded images version 
list_of_test_img_matrixes = []

for i in range(len(fulllistlabels)): # len = 825
    true_label_num = fulllistlabels[i]
    if true_label_num == 0:
        classnamed = "Drones"
    else:
        classnamed = "Birds"
    path_to_img = test_path + classnamed + "/" + fulllistnames[i]

    # img = cv2.imread(path_to_img)
    # RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # smallimgb = cv2.resize(RGB_img, (180,180),interpolation=cv2.INTER_CUBIC)

    img = tf.keras.utils.load_img(path_to_img, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) #  Create a batch
    # print(img_array.shape)

    # img_path = test_path + classnamed + "/" + fulllistnames[i]
    # img = tf.keras.utils.load_img(img_path, target_size=(img_height, img_width))
    # img_array = tf.keras.utils.img_to_array(img)
    # img_array = tf.expand_dims(img_array, 0) #  Create a batch
    # predictions = new_model.predict(img_array)

    # normsmallimg = smallimgb/ 255.
    list_of_test_img_matrixes.append([img_array, true_label_num]) #?????

In [ ]:
false_class_num =0
for i in range(len(fulllistlabels)): # len = 825

    # img = tf.keras.utils.load_img(img_path, target_size=(img_height, img_width))
    test_img = list_of_test_img_matrixes[i][0]
    label_num = list_of_test_img_matrixes[i][1]
    # plt.imshow(test_img)
    # plt.show()
    # print (np.array(test_img).shape)
    # print (label_num)

    # img_array = tf.keras.utils.img_to_array(test_img)
    # img_array = tf.expand_dims(img_array, 0) #  Create a batch

    predictions = new_model.predict(test_img)
    tf.keras.backend.clear_session()
    score = tf.nn.softmax(predictions[0])
    if label_num == np.argmax(score):
        false_class_num += 1
    print(
    "This image most likely belongs to {} with a {:.2f}% confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

print("test_accuracy = ",false_class_num/len(fulllistlabels))
